In [0]:
# clear VM completely
! kill -9 -1

In [1]:
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

from google.colab import auth
auth.authenticate_user()
with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

TPU address is grpc://10.123.151.250:8470


W0821 09:26:27.837970 140154367981440 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 5164580629985135382),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 11837996157355801182),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 6213735751976984636),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 1257301642428864053),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 3896444144470866598),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 11557949942989894400),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 9787055620788696470),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 8556736308082469437),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 9186177757265636374),
 _DeviceAttributes(/job:tpu_work

In [2]:
!pip install sentencepiece

     |████████████████████████████████| 1.0MB 3.5MB/s 


In [3]:
!gsutil cp gs://<YOUR_BUCKET_NAME>/th_wiki_bpe.zip .

Copying gs://no-idea-what-to-name-bucket/th_wiki_bpe.zip...
/ [1 files][488.7 KiB/488.7 KiB]                                                
Operation completed over 1 objects/488.7 KiB.                                    


In [4]:
!unzip th_wiki_bpe.zip

Archive:  th_wiki_bpe.zip
   creating: th_wiki_bpe/
  inflating: th_wiki_bpe/th.wiki.bpe.op25000.model  
  inflating: th_wiki_bpe/th.wiki.bpe.op25000.vocab  


In [5]:
import sys

!test -d bert_repo || git clone https://github.com/ThAIKeras/bert.git bert_thai
if not 'bert_thai' in sys.path:
  sys.path += ['bert_thai']

# import python modules defined by BERT
import modeling
import optimization
import run_classifier
# import run_classifier_with_tfhub
import tokenization

# import tfhub 
# import tensorflow_hub as hub

Cloning into 'bert_thai'...
remote: Enumerating objects: 275, done.
remote: Total 275 (delta 0), reused 0 (delta 0), pack-reused 275
Receiving objects: 100% (275/275), 197.94 KiB | 2.71 MiB/s, done.
Resolving deltas: 100% (151/151), done.


W0821 09:26:51.702592 140154367981440 deprecation_wrapper.py:119] From bert_thai/optimization.py:84: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [6]:
!git clone https://github.com/wongnai/wongnai-corpus.git

Cloning into 'wongnai-corpus'...
remote: Enumerating objects: 118, done.
remote: Total 118 (delta 0), reused 0 (delta 0), pack-reused 118
Receiving objects: 100% (118/118), 39.65 MiB | 32.07 MiB/s, done.
Resolving deltas: 100% (42/42), done.


In [7]:
%cd wongnai-corpus/review/

/content/wongnai-corpus/review


In [8]:
!unzip review_dataset.zip

Archive:  review_dataset.zip
 extracting: sample_submission.csv   
  inflating: test_file.csv           
  inflating: w_review_train.csv      
   creating: __MACOSX/
  inflating: __MACOSX/._sample_submission.csv  
  inflating: __MACOSX/._test_file.csv  
  inflating: __MACOSX/._w_review_train.csv  


In [9]:
%cd ../..

/content


In [10]:
!ls th_wiki_bpe/

th.wiki.bpe.op25000.model  th.wiki.bpe.op25000.vocab


In [11]:
TASK = 'wongnai'
TASK_DATA_DIR = '/content/wongnai-corpus/review/'
OUTPUT_DIR = 'gs://<YOUR_BUCKET_NAME>/wongnai_bert_tpu'
tokenizer = tokenization.ThaiTokenizer(vocab_file='/content/th_wiki_bpe/th.wiki.bpe.op25000.vocab', spm_file='/content/th_wiki_bpe/th.wiki.bpe.op25000.model')

W0821 09:27:00.549983 140154367981440 deprecation_wrapper.py:119] From bert_thai/tokenization.py:74: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.



In [12]:
TRAIN_BATCH_SIZE = 128
EVAL_BATCH_SIZE = 8
PREDICT_BATCH_SIZE = 8
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 10
MAX_SEQ_LENGTH = 128
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 1000
SAVE_SUMMARY_STEPS = 500

tf.logging.set_verbosity(tf.logging.INFO)

processors = {
  "cola": run_classifier.ColaProcessor,
  "mnli": run_classifier.MnliProcessor,
  "mrpc": run_classifier.MrpcProcessor,
  "wongnai": run_classifier.WongnaiProcessor,
}
processor = processors[TASK.lower()]()
label_list = processor.get_labels()

# Compute number of train and warmup steps from batch size
train_examples = processor.get_train_examples(TASK_DATA_DIR)
num_train_steps = int(len(train_examples) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

# Setup TPU related config
tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)
NUM_TPU_CORES = 8
ITERATIONS_PER_LOOP = 1000

def get_run_config(output_dir):
  return tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=output_dir,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=ITERATIONS_PER_LOOP,
        num_shards=NUM_TPU_CORES,
        per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))


W0821 09:27:00.753659 140154367981440 deprecation_wrapper.py:119] From bert_thai/run_classifier.py:386: The name tf.gfile.Open is deprecated. Please use tf.io.gfile.GFile instead.



In [0]:
bert_config = modeling.BertConfig.from_json_file('gs://<YOUR_BUCKET_NAME>/bert_base_th/bert_config.json')

model_fn = run_classifier.model_fn_builder(
  bert_config=bert_config,
  num_labels=len(label_list),
  init_checkpoint='gs://<YOUR_BUCKET_NAME>/bert_base_th/model.ckpt',
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps,
  use_tpu=True,
  use_one_hot_embeddings=True
)

In [14]:
estimator = tf.contrib.tpu.TPUEstimator(
  use_tpu=True,
  model_fn=model_fn,
  config=get_run_config(OUTPUT_DIR),
  train_batch_size=TRAIN_BATCH_SIZE,
  eval_batch_size=EVAL_BATCH_SIZE,
  predict_batch_size=PREDICT_BATCH_SIZE
)

W0821 09:27:05.764487 140154367981440 estimator.py:1984] Estimator's model_fn (<function model_fn_builder.<locals>.model_fn at 0x7f77f2c5ce18>) includes params argument, but params are not passed to Estimator.
I0821 09:27:05.770701 140154367981440 estimator.py:209] Using config: {'_model_dir': 'gs://no-idea-what-to-name-bucket/wongnai_bert_tpu', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.123.151.250:8470"
    }
  }
}
isolate_session_state: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.C

In [15]:
train_file = os.path.join(OUTPUT_DIR, "train.tf_record")
run_classifier.file_based_convert_examples_to_features(
    train_examples, label_list, MAX_SEQ_LENGTH, tokenizer, train_file)
tf.logging.info("***** Running training *****")
tf.logging.info("  Num examples = %d", len(train_examples))
tf.logging.info("  Batch size = %d", TRAIN_BATCH_SIZE)
tf.logging.info("  Num steps = %d", num_train_steps)
train_input_fn = run_classifier.file_based_input_fn_builder(
    input_file=train_file,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=True)
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)

W0821 09:27:05.793981 140154367981440 deprecation_wrapper.py:119] From bert_thai/run_classifier.py:508: The name tf.python_io.TFRecordWriter is deprecated. Please use tf.io.TFRecordWriter instead.

W0821 09:27:05.797576 140154367981440 deprecation_wrapper.py:119] From bert_thai/run_classifier.py:512: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.

I0821 09:27:05.799546 140154367981440 run_classifier.py:512] Writing example 0 of 40000
I0821 09:27:05.817574 140154367981440 run_classifier.py:487] *** Example ***
I0821 09:27:05.818720 140154367981440 run_classifier.py:488] guid: train-0
I0821 09:27:05.820106 140154367981440 run_classifier.py:490] tokens: [CLS] ▁ร้าน อาหาร ใหญ่มาก กก กก กก ▁ <unk> เลี้ยว เข้ามา เจอ ห้อง น้ํา ก่อน เลย ▁เอ อ แปลก ดี ▁ <unk> ห้อง ทาน หลักๆ อยู่ ชั้น ▁ 2 ▁มี กาแฟ ▁น้ํา ผึ้ง ▁ซึ่ง ก็ แค่ เอ าน้ํา ผึ้ง มารา ด ▁แพ ง เวอร์ ▁ อย่า สั่ง เลย ▁ <unk> ลา บ ไข่ ต้ม ▁ไข ่ม ัน คาว อะ ▁เลย ไม่ ประทับใจ เท่า ไหร ่ <unk> ทอด มัน หัว ปลี 